# 학습데이터, 테스트데이터 만들기

In [20]:
import pandas as pd
import numpy as np
import torch
from tqdm.auto import tqdm
import random
import os

def reset_seeds(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True

device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cuda'

In [21]:
train_ft = pd.read_excel('train_tmp.xlsx')
test_ft = pd.read_excel('test_tmp.xlsx')

train_ft.shape,test_ft.shape

((4112, 8), (1028, 8))

In [22]:
train_status = train_ft.loc[:,['리뷰','제품상태_긍부정']].copy()
train_status = train_status[train_status['제품상태_긍부정'].isin([-1,1])]

test_status = test_ft.loc[:,['리뷰','제품상태_긍부정']].copy()
test_status = test_status[test_status['제품상태_긍부정'].isin([-1,1])]

train_status.shape,test_status.shape

((339, 2), (85, 2))

In [23]:
train_status['제품상태_긍부정'].value_counts() 
# -1의 비율이 훨씬 더 높은 수준임을 알 수 있음 

제품상태_긍부정
-1    292
 1     47
Name: count, dtype: int64

In [24]:
train_status.columns

Index(['리뷰', '제품상태_긍부정'], dtype='object')

In [25]:
train_arr = train_status['리뷰'].to_numpy()
test_arr = test_status['리뷰'].to_numpy()

train_arr.shape,test_arr.shape

((339,), (85,))

In [26]:
target = train_status['제품상태_긍부정'].to_numpy().reshape(-1,1)
target = np.where(target == -1,0,target)

target_test = test_status['제품상태_긍부정'].to_numpy().reshape(-1,1)
target_test = np.where(target_test == -1,0,target_test)

target = target.astype('int64')
target_test = target_test.astype('int64')

target.shape,target_test.shape

((339, 1), (85, 1))

In [27]:
target.mean()

0.13864306784660768

# 사전학습 모델 선정

In [28]:
model_name = 'kykim/bert-kor-base'

In [29]:
from transformers import AutoTokenizer, AutoModel

In [30]:
model = AutoModel.from_pretrained(model_name)

/home/lab01/.local/lib/python3.10/site-packages/torch/_utils.py:776: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


In [31]:
tokenizer = AutoTokenizer.from_pretrained(model_name)

# 사전학습모델을 바탕으로 학습 실행

In [32]:
class ReviewDataset(torch.utils.data.Dataset):
    def __init__(self, tokenizer, x, y=None):
        self.tokenizer = tokenizer
        self.x = x
        self.y = y
    def __len__(self):
        return len(self.x)
    def __getitem__(self, idx):
        item = {}
        item["x"] = self.get_tokenizer(self.x[idx])
        if self.y is not None:
            item["y"] = torch.Tensor(self.y[idx])
        return item
    def get_tokenizer(self, text):
        x = self.tokenizer(text, padding="max_length", truncation=True)
        for k, v in x.items():
            x[k] = torch.tensor(v)
        return x

In [33]:
dt = ReviewDataset(tokenizer, train_arr, target)
dl = torch.utils.data.DataLoader(dt, batch_size=2, shuffle=False)
batch = next(iter(dl))
batch

{'x': {'input_ids': tensor([[    2, 18068, 14289,  ...,     0,     0,     0],
         [    2, 14668, 25103,  ...,     0,     0,     0]]), 'token_type_ids': tensor([[0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0]])},
 'y': tensor([[0.],
         [1.]])}

In [34]:
class Net(torch.nn.Module):
    def __init__(self, model_name):
        super().__init__()
        self.pre_model = AutoModel.from_pretrained(model_name)
        self.fc_out = torch.nn.Linear( self.pre_model.config.hidden_size, 1)

    def forward(self, x):
        x = self.pre_model(**x)
        # x[0]: 모든 시점의 히든출력 batch, seq, features
        # x[1]: CLS 토큰의 히든출력 batch, features
        return self.fc_out(x[1])

In [35]:
model = Net(model_name)
model(batch["x"])

tensor([[-0.0146],
        [-0.0302]], grad_fn=<AddmmBackward0>)

In [36]:
def train_loop(dataloader, model, loss_fn, optimizer, device):
    epoch_loss = 0
    model.train() # 학습 모드
    for batch in tqdm(dataloader):
        pred = model( batch["x"].to(device) )
        loss = loss_fn( pred, batch["y"].to(device) )

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        epoch_loss += loss.item()

    epoch_loss /= len(dataloader)
    return epoch_loss

In [37]:
@torch.no_grad()
def test_loop(dataloader, model, loss_fn, device):
    epoch_loss = 0
    pred_list = []
    act_func = torch.nn.Sigmoid()
    model.eval() # 평가 모드
    for batch in tqdm(dataloader):
        pred = model( batch["x"].to(device) )
        if batch.get("y") is not None:
            loss = loss_fn( pred, batch["y"].to(device) )
            epoch_loss += loss.item()

        pred = act_func(pred) # logit 값을 확률로 변환
        pred = pred.to("cpu").numpy() # cpu 이동후 ndarray 로변환
        pred_list.append(pred)

    epoch_loss /= len(dataloader)
    pred = np.concatenate(pred_list)
    return epoch_loss, pred

In [38]:
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score

n_splits = 5
cv = KFold(n_splits, shuffle=True, random_state=42)

batch_size = 8 # 배치 사이즈
loss_fn = torch.nn.BCEWithLogitsLoss() # 손실 객체
epochs = 100 # 최대 가능한 에폭수

In [39]:
is_holdout = False
reset_seeds(42) # 재현을 위해 시드고정
best_score_list = []
for i, (tri, vai) in enumerate( cv.split(train_arr) ):
    # 학습용 데이터로더 객체
    train_dt = ReviewDataset(tokenizer, train_arr[tri], target[tri])
    train_dl = torch.utils.data.DataLoader(train_dt, batch_size=batch_size, shuffle=True)

    # 검증용 데이터로더 객체
    valid_dt = ReviewDataset(tokenizer, train_arr[vai], target[vai])
    valid_dl = torch.utils.data.DataLoader(valid_dt, batch_size=batch_size, shuffle=False)

    # 모델 객체와 옵티마이저 객체 생성
    model = Net(model_name).to(device)
    optimizer = torch.optim.Adam( model.parameters(), lr=2e-5 )

    best_score = 0 # 현재 최고 점수
    patience = 0 # 조기 종료 조건을 주기 위한 변수
    for epoch in range(epochs):
        train_loss = train_loop(train_dl, model, loss_fn, optimizer, device)
        valid_loss, pred = test_loop(valid_dl, model, loss_fn, device)

        pred = (pred > 0.5).astype(int) # 이진분류 문제에서 클래스 번호 결정
        score = f1_score(target[vai], pred,average="macro")

        print(train_loss, valid_loss, score)
        if score > best_score:
            best_score = score # 최고 점수 업데이트
            patience = 0
            torch.save(model.state_dict(), f"model_{i}.pth") # 최고 점수 모델 가중치 저장

        patience += 1
        if patience == 5:
            break

    print(f"{i}번째 폴드 최고 정확도: {best_score}")
    best_score_list.append(best_score)

    if is_holdout:
        break

  0%|          | 0/34 [00:00<?, ?it/s]

KeyboardInterrupt: 

# 테스트 데이터 예측하기 

In [103]:
test_status['리뷰']

34     가격대비 퀄리티 너무 괜찮아요!! 컬러별로 샀어요 몇 일 입었는데 보풀일어나지도 않...
45     제가 원하던 핏과 길이 라서 정말만족합니다.\n다만 옷이 생각보다 무겁고, 처음에 ...
71     일단 후기 보고 비침이 심하단 건 알아서 안에 나시 입을 거 생각하고 디자인이랑 핏...
73     냄새가 좀 나서 세탁하고 몇칠 말렸더니   괜찮네요\n여성여성 하니 이뻐요.\n싼 ...
101    핏 너무 이쁘고 가볍고 좋은데 뜯었을때 냄새가 좀 심해용\n그리고 생각보다 너무너무...
                             ...                        
916    처음 배송 왔을때는 잘 몰랐는데\n냄새 때문에 한번도 않입고 단독세탁으로 빨았는데 ...
921    흠냐 일단 배송 2주나 기다렸는데 옷에 얼룩이 묻어서 왔구요 ^^ \n택배 다시 올...
930    티가 너~무 부드럽고 좋은데 이런.. 오염?이 있어요ㅠ 여행직전에 빨리 받으려고 산...
932    생각보다 여름에 입기에는 두꺼울 것 같아요.  옷 자체도 그저 그런데 하자있는 상품...
991    한여름에 입기는 조금 더운 느낌이에요! 하지만 그만큼 탄탄한 원단이곤해요~\n가슴쪽...
Name: 리뷰, Length: 85, dtype: object

In [117]:
pd.Series(['보풀이 너무 많아요','가격이 너무 비싸요']).to_numpy()

array(['보풀이 너무 많아요', '가격이 너무 비싸요'], dtype=object)

In [151]:
def tmp_tmp(sentence):
    if isinstance(sentence, pd.Series):
        train_arr = sentence.to_numpy()
        return train_arr
    else:
        train_arr = pd.Series(sentence).to_numpy()
        return train_arr

In [152]:
tmp_tmp(['안녕하세요','양태성입니다. 어렵네요'])

array(['안녕하세요', '양태성입니다. 어렵네요'], dtype=object)

In [124]:
def example(*sentence):
    temp = pd.Series(sentence).to_numpy()
    return temp

In [127]:
example('너무 어렵다','매우 어렵다')

array(['너무 어렵다', '매우 어렵다'], dtype=object)

In [89]:
tmp = np.array(['보풀이 너무 많아서 짜증나요!','너무 아름다워요'],dtype='object')

In [94]:
test_dt = ReviewDataset(tokenizer,test_arr)
test_dl = torch.utils.data.DataLoader(test_dt, batch_size=2, shuffle=False)
batch =next(iter(test_dl))
batch['x']

{'input_ids': tensor([[    2, 17384, 17014,  ...,     0,     0,     0],
        [    2, 14137, 30253,  ...,     0,     0,     0]]), 'token_type_ids': tensor([[0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]])}

In [109]:
test_dt = ReviewDataset(tokenizer,tmp)
test_dl = torch.utils.data.DataLoader(test_dt, batch_size=1, shuffle=False)
batch =next(iter(test_dl))
batch['x']

ValueError: text input must be of type `str` (single example), `List[str]` (batch or single pretokenized example) or `List[List[str]]` (batch of pretokenized examples).

In [44]:
result = tokenizer(test_arr[0], padding="max_length", truncation=True)
input_ids_tensor = torch.tensor(result['input_ids']).unsqueeze(0)
token_type_ids_tensor = torch.tensor(result['token_type_ids']).unsqueeze(0)
attention_mask_tensor = torch.tensor(result['attention_mask']).unsqueeze(0)

In [45]:
result = {
    'input_ids': input_ids_tensor,
    'token_type_ids': token_type_ids_tensor,
    'attention_mask': attention_mask_tensor
}

result

{'input_ids': tensor([[    2, 17384, 17014, 14000, 18434,  2005,  2005, 14273, 15505, 16697,
           4145,  5929,  5938, 14172, 30876, 41206, 14260, 14169, 17407, 14000,
           5957, 15312, 33681,  8114, 22175,  8666,  8078, 14000, 21590, 16639,
          14848, 16685, 35003, 20216,  8073, 14835, 14251, 23577,  8114,     3,
              0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
              0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
              0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
              0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
              0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
              0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
              0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
              0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
              0,     0,    

In [87]:
pred_list = []
for i in range(n_splits):
    model = Net(model_name).to(device)
    state_dict = torch.load(f"model_{i}.pth")
    model.load_state_dict(state_dict)

    _, pred = test_loop(test_dl, model, loss_fn, device)

    pred_list.append(pred)
    if is_holdout:
        break

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

In [88]:
pred = np.mean(pred_list, axis=0)
pred = (pred > 0.5).astype(int)
pred[0]

array([0])

In [38]:
score = accuracy_score(target_test, pred)
score

0.9411764705882353